Indexing Metrics
Indexing everything will slow down the Inserts and make Inserts expensive
Composite Index can help you with Order BY and WHERE with many filters
Global Secondary Indexes

Indexing Metrics
select * from c 
where c.OwnerUserId = 200 
and c.PostType ='Question' 
and c.Score > 10

GSI = Cross Partition's nightmare
Data gets sync automatically
GSI has its own structure (pkey, data model, indexing)


